# Đồ Án Cuối Kì Môn Khoa Học Dữ Liệu


**Giới thiệu thành viên**

- *Họ tên:* **Trần Ngọc Tịnh**       
  *MSSV:* **18120597**
  
  
  
  
- *Họ tên:* **Nguyễn Ngọc Năng Toàn**       
  *MSSV:* **18120600**

## Import 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

# You can also import other things ...
# YOUR CODE HERE (OPTION)

## Đọc dữ liệu từ file csv

In [2]:
data_df = pd.read_csv('data\data.csv', 
                      index_col=0) # Cho cột index là cột 0 (PassengerId)
data_df.head()

,confirmed,recovered,deaths,population,sq_km_area,life_expectancy,elevation_in_meters,continent,abbreviation,location,iso,capital_city,lat,long,updated
Country,,,,,,,,,,,,,,,
Afghanistan,51526,41727,2191,35530081.0,652090.0,45.9,NaN,Asia,AF,Southern and Central Asia,4.0,Kabul,33.93911,67.709953,2021/01/01 14:22:58+00
Albania,58316,33634,1181,2930187.0,28748.0,71.6,NaN,Europe,AL,Southern Europe,8.0,Tirana,41.15330,20.168300,2021/01/01 14:22:58+00
Algeria,99610,67127,2756,41318142.0,2381741.0,69.7,800,Africa,DZ,Northern Africa,12.0,Alger,28.03390,1.659600,2021/01/01 14:22:58+00
Andorra,8049,7432,84,76965.0,468.0,83.5,"1,996",Europe,AD,Southern Europe,20.0,Andorra la Vella,42.50630,1.521800,2021/01/01 14:22:58+00
Angola,17553,11044,405,29784193.0,1246700.0,38.3,"1,112",Africa,AO,Central Africa,24.0,Luanda,-11.20270,17.873900,2021/01/01 14:22:58+00


## Khám phá dữ liệu 

In [3]:
data_df.index.duplicated().sum()

0

In [4]:
data_df['deaths'].dtype

dtype('int64')

In [5]:
data_df['deaths'].isna().sum()

0

In [6]:
data_df['deaths'].value_counts(normalize=True) * 100

0       7.558140
63      1.744186
104     1.162791
7576    1.162791
1468    1.162791
          ...   
1499    0.581395
671     0.581395
1181    0.581395
410     0.581395
119     0.581395
Name: deaths, Length: 150, dtype: float64

## Tách các tập

In [7]:
y_sr = data_df["deaths"] # sr là viết tắt của series
X_df = data_df.drop("deaths", axis=1)

In [8]:
train_X, test_X, train_y, test_y = train_test_split(X_df, y_sr, test_size=0.3, random_state=0)

## Tiền xử lí ( tập huấn luyện )

### Dữ liệu trước khi xử lí

In [9]:
train_X.shape

(120, 14)

In [10]:
test_X.shape

(52, 14)

### Class Tiền xử lí

In [11]:
class ColAdderDropper(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        df = X_df.copy()
        unused_cols = ['abbreviation', 'capital_city', 'updated']
        df=df.drop(unused_cols,axis=1)
        df=df.astype({'confirmed':'float64','recovered':'float64'})
        df['elevation_in_meters'] = pd.to_numeric(df['elevation_in_meters'],errors='coerce')
        return df

### Fit & Transform

In [12]:
col_adderdropper = ColAdderDropper()
col_adderdropper.fit(train_X)
col_adderdropper.transform(train_X).head()



,confirmed,recovered,population,sq_km_area,life_expectancy,elevation_in_meters,continent,location,iso,lat,long
Country,,,,,,,,,,,
Cyprus,22019.0,2057.0,1179551.0,9251.0,76.7,91.0,Asia,Middle East,196.0,35.126400,33.429900
Tanzania,509.0,183.0,57310019.0,883749.0,52.3,NaN,Africa,Eastern Africa,834.0,-6.369028,34.888822
Botswana,14805.0,12136.0,2291661.0,581730.0,39.3,NaN,Africa,Southern Africa,72.0,-22.328500,24.684900
Morocco,439193.0,407504.0,35739580.0,446550.0,69.1,909.0,Africa,Northern Africa,504.0,31.791700,-7.092600
Togo,3633.0,3393.0,7797694.0,56785.0,54.7,236.0,Africa,Western Africa,768.0,8.619500,0.824800


### Kiểu dữ liệu sau xử lí

In [13]:
col_adderdropper.transform(train_X).dtypes

confirmed              float64
recovered              float64
population             float64
sq_km_area             float64
life_expectancy        float64
elevation_in_meters    float64
continent               object
location                object
iso                    float64
lat                    float64
long                   float64
dtype: object

In [14]:
col_adderdropper.transform(train_X).shape

(120, 11)

### Giá trị NAN

In [15]:
col_adderdropper.transform(train_X).isna().sum()

confirmed               0
recovered               0
population              2
sq_km_area              2
life_expectancy         3
elevation_in_meters    40
continent               2
location                2
iso                     3
lat                    11
long                   11
dtype: int64

### Đảm bảo tập train_X không bị thay đổi 

In [16]:
train_X.shape

(120, 14)

### Điền giá trị thiếu

In [17]:
nume_cols=['confirmed','recovered','population','sq_km_area','life_expectancy','iso','lat','long','elevation_in_meters']
cate_cols=['continent','location']

mean = SimpleImputer(strategy='mean')
mode = SimpleImputer(strategy='most_frequent') 
mode_encode = make_pipeline(mode,OneHotEncoder(handle_unknown='ignore'))

colTransform = ColumnTransformer([('mean_for_nume_cols',mean, nume_cols),\
                                 ('mode_encode_for_cate_cols',mode_encode,cate_cols)])
colNormalize = StandardScaler()
preprocess_pipeline = make_pipeline(col_adderdropper,colTransform,colNormalize)

preprocessed_train_X=preprocess_pipeline.fit_transform(train_X)

preprocess_pipeline

Pipeline(steps=[('coladderdropper', ColAdderDropper()),
                ('columntransformer',
                 ColumnTransformer(transformers=[('mean_for_nume_cols',
                                                  SimpleImputer(),
                                                  ['confirmed', 'recovered',
                                                   'population', 'sq_km_area',
                                                   'life_expectancy', 'iso',
                                                   'lat', 'long',
                                                   'elevation_in_meters']),
                                                 ('mode_encode_for_cate_cols',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['continent', 'location'])])),
                ('standardscaler', StandardScaler())])